# News Classification Model using Keras

In [1]:
# import all libraries needed

import pandas as pd
from pandas import read_csv
import os
import spacy
import numpy as np
import matplotlib.pyplot as plt
import langid

# nltk used for parsing and cleaning text
import nltk
import unicodedata
import string
nltk.download('punkt')
nltk.download('stopwords')
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from nltk.probability import FreqDist
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer
from difflib import SequenceMatcher
from scipy import spatial
from itertools import combinations

# used to acccess the sql database
import pymysql
# library that helps turn dataframes into sql tables
from sqlalchemy import create_engine


import sklearn as sk
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score, confusion_matrix, matthews_corrcoef
from sklearn.linear_model import LogisticRegression

import pickle

## for deep learning
import tensorflow as tf
from tensorflow.keras.preprocessing.text import one_hot, Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Embedding, Input, LSTM, Conv1D, MaxPool1D, Bidirectional
from tensorflow.keras.models import Model
from tensorflow.keras import models, layers, preprocessing as kprocessing
from tensorflow.keras import backend as K

import warnings
warnings.filterwarnings('ignore')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/arminberger/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/arminberger/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
current_dir = os.getcwd()
current_dir 

'/Users/arminberger/Documents/GitHub/iteration_1_model'

In [4]:
# create connection
connection = pymysql.connect(host='news-data-rdb.cqsnaejqwcpu.ap-southeast-2.rds.amazonaws.com',
                             user='admin',
                             password='badpassword1',
                             db='news_data')

In [7]:
# SQL query to retreive our data for the model
SQL_Query = pd.read_sql_query(
    '''select
        *
        from new_table''', connection)

In [8]:
SQL_Query

,news_id,text,label
0,0,house aide comey letter jason chaffetz tweeted...,1
1,1,flynn hillary clinton woman campus breitbart f...,0
2,2,truth fired truth fired october tension intell...,1
3,3,civilians killed single airstrike identified v...,1
4,4,iranian woman jailed fictional unpublished sto...,1
...,...,...,...
941403,941403,buzzfeed year awkward convo dude texting trump...,1
941404,941404,littlebytesnews tcot news variety facebook too...,1
941405,941405,signsinyork getting right company logo busines...,1
941406,941406,latest obama affirms continuity ties canada ht...,1


In [10]:
# save the data rereived in a query as a pandas dataframe
news_sql_df = pd.DataFrame(SQL_Query, columns=['news_id', 'text', 'label'])

In [11]:
news_sql_df 

,news_id,text,label
0,0,house aide comey letter jason chaffetz tweeted...,1
1,1,flynn hillary clinton woman campus breitbart f...,0
2,2,truth fired truth fired october tension intell...,1
3,3,civilians killed single airstrike identified v...,1
4,4,iranian woman jailed fictional unpublished sto...,1
...,...,...,...
941403,941403,buzzfeed year awkward convo dude texting trump...,1
941404,941404,littlebytesnews tcot news variety facebook too...,1
941405,941405,signsinyork getting right company logo busines...,1
941406,941406,latest obama affirms continuity ties canada ht...,1


In [12]:
# split data into test and train 
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(news_sql_df.text, news_sql_df.label, test_size = 0.1)

In [13]:
vectorizer = TfidfVectorizer(analyzer = 'word',
                             input = 'content',
                            lowercase = True,
                            token_pattern = '(?u)\\b\\w\\w+\\b',
                            min_df = 3,
                            ngram_range = (1,2))

In [15]:
# vectorize the training text input 
vectorizer.fit_transform(news_sql_df.text.to_list())

KeyboardInterrupt: 

In [ ]:
# vectorize the training text input 
x_train = vectorizer.transform(x_train)
x_test = vectorizer.transform(x_test)

## Model

In [ ]:
# Sequential Model
model = Sequential()

# embeddidng layer to do PCA 
# total_words = defined the total number of vocabs 
model.add(Embedding(total_words, output_dim = 1000))
# model.add(Embedding(total_words, output_dim = 240))


# Bi-Directional RNN and LSTM
model.add(Bidirectional(LSTM(1000)))

# Dense layers
model.add(Dense(1000, activation = 'relu'))
model.add(Dense(1, activation= 'sigmoid'))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
model.summary()

In [ ]:
# turn binary labels intp a numpy array
y_train = np.asarray(y_train)

In [ ]:
# train the model
#  validation_split = 0.1, use 10% of the data for cross validation
model.fit(x_train, y_train, batch_size = 64, validation_split = 0.1, epochs = 5)

In [ ]:
# make prediction
pred = model.predict(x_test)

In [ ]:
# if the predicted value is >0.5 it is real else it is fake
prediction = []
for i in range(len(pred)):
    if pred[i].item() > 0.5:
        prediction.append(1)
    else:
        prediction.append(0)

In [ ]:
# getting the accuracy
from sklearn.metrics import accuracy_score

accuracy = accuracy_score(list(y_test), prediction)

print("Model Accuracy : ", accuracy)